In [ ]:
## install libraries

import pandas as pd
import numpy as np
import datetime
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [ ]:
## load files

dsli = []
for i in range(1,57):
  file_name = '/content/part_' + str(i) + ".csv"
  ds = pd.read_csv(file_name)
  dsli.append(ds)
dataset = pd.concat(dsli,axis=0, ignore_index=True)

stores = pd.read_csv('/content/stores.csv')

In [ ]:
##convert datetime in to string 

dataset['date'] = dataset['utc_timestamp'].apply(lambda x: (datetime.datetime.fromtimestamp(x / 1e3)).strftime("%m/%d/%Y") )
dataset.drop("utc_timestamp",axis =1, inplace = True)

In [ ]:
###Affinity Data Consolidation Attempt

affinity = ['addidas','apple','bmw','employed','female','h_&_m','high_income','honda','job_seeking','low_income','male','mercedes-benz','middle_income']
affinity_df = pd.DataFrame(columns = affinity)

for i in affinity:
  affinity_df[i] = pd.read_csv('/content/'+i+'.csv',squeeze = True)


In [ ]:
## this is not working as expected. 
## i wanted to check all device ids in each affinity group however the same list continues to be generated

def affinity_pairer(x):
  l = []
  for i in dataset['device_id']:
    if i in affinity_df[x]:
      l.append(1)
    else:
      l.append(0)
  print(len(l))
  return l
  

dataset['addidas'] = affinity_pairer('addidas')

device_id_list = dataset['device_id']
l=[]
for i in device_id_list:
  if i in affinity_df['addidas']:
    l.append('yes')

  else:
    l.append('false')
print(l)


2240000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
affinity_df

,addidas,apple,bmw,employed,female,h_&_m,high_income,honda,job_seeking,low_income,male,mercedes-benz,middle_income
0,90618,58923,160780,128202,85001,125654,9,161033,64710,5,1,76363,2
1,97745,141158,78690,72506,85002,31465,14,169431,161402,10,2,133934,3
2,45843,164532,114955,76600,85003,170616,19,92534,17976,15,3,47673,6
3,164807,88589,129177,116724,85004,34023,24,166431,6752,20,4,120911,7
4,106524,140769,103507,24898,85005,83629,29,38117,137026,25,5,11361,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6770,112211,368,151089,118432,91771,130963,33859,59903,116937,33855,6771,114289,11286
6771,112457,39709,57341,117036,91772,86427,33864,112714,134804,33860,6772,283,11287
6772,77740,136922,106797,64245,91773,142040,33869,168415,129429,33865,6773,38223,11288
6773,50930,143039,70049,39194,91774,34711,33874,101600,113307,33870,6774,160752,11291


In [ ]:
dataset

,device_id,lat,lon,date,addidas
0,21166,52.554763,13.453171,01/01/2021,0
1,22514,52.514669,13.190656,01/01/2021,0
2,19025,52.431905,13.470953,01/01/2021,0
3,21728,52.502707,13.365311,01/01/2021,0
4,5646,52.479403,13.423215,01/01/2021,1
...,...,...,...,...,...
2239995,170705,52.464065,13.434172,01/21/2021,0
2239996,23107,52.323267,13.380991,01/21/2021,0
2239997,27126,52.522315,13.199622,01/21/2021,0
2239998,36278,52.539499,13.291320,01/21/2021,0


In [ ]:
## function to convert polygon data into suitable format
def polylist(x):
  i = 0
  l = []
  polygon_list = x[x.find("(")+2:x.find(")")]
  polygon_list = polygon_list.split(',')

  while i < len(polygon_list):
    l.append(list(map(float,polygon_list[i].split())))
    
    i = i+1
  return l

### convert long & lat to point

i = 0
point_list = []
while i < dataset.shape[0]:
  point1 = Point(dataset['lon'][i],dataset['lat'][i])
  point_list.append(point1)
  i +=1

dataset['point'] = point_list
stores['polygon_points'] = stores['wkt'].apply(lambda x: polylist(x) )

stores.drop('wkt', axis =1, inplace=True)
dataset.drop(['lon','lat'],axis=1, inplace = True)

#### creates polygon column

i = 0
polygon_list = []
while i < stores.shape[0]:
  polygon_list.append(Polygon(stores['polygon_points'][i]))
  i +=1

stores['polygon_shape'] = polygon_list

,device_id,date,addidas,point
0,21166,01/01/2021,0,POINT (13.4531708 52.55476280000001)
1,22514,01/01/2021,0,POINT (13.1906557 52.5146689)
2,19025,01/01/2021,0,POINT (13.470953 52.43190500000001)
3,21728,01/01/2021,0,POINT (13.365311 52.502707)
4,5646,01/01/2021,1,POINT (13.4232152 52.4794033)
...,...,...,...,...
119953,48602,01/01/2021,0,POINT (13.3622978 52.5675236)
119954,50184,01/01/2021,0,POINT (13.493236 52.532013)
119965,49292,01/01/2021,0,POINT (13.466106 52.473569)
119974,48538,01/01/2021,0,POINT (13.2070302 52.6046417)


In [ ]:
## checks points in polygons and creates new df from the point data that matched

final_data = pd.DataFrame(columns = ['date','stores','store_id','total signals','unique visitors'])

j =  0 

while j < stores.shape[0]:

  i = 0
  visitors = []
  new_row = {'date':dataset['date'],'stores':stores['store_name'][j],'store_id':stores['store_id'][j],'total signals' : 0,'unique visitors':0}

  while i < len(dataset['point']):

    if (stores['polygon_shape'][j].contains(dataset['point'][i])):

      if dataset['device_id'][i] not in visitors:
        new_row['unique visitors'] +=1
        visitors.append(dataset['device_id'][i])

      new_row['total signals'] += 1
      i +=1

    else:
      i+=1
      continue

  if new_row['total signals'] > 0:
    final_data = final_data.append(new_row,ignore_index=True)
      
  print(j)
  print(new_row)
  j+=1

final_data

0
{'date': '01/01/2021', 'stores': "McDonald's", 'store_id': 'place_1', 'total signals': 7, 'unique visitors': 6}


KeyboardInterrupt: ignored

In [ ]:
## checks points in polygons and creates new df from the point data that matched

unique_dates = dataset['date'].unique()

final_data = pd.DataFrame(columns = ['date','stores','store_id','total signals','unique visitors'])

j =  0 

while j < stores.shape[0]:
 
  for date in unique_dates:

    date_filtered = dataset[dataset['date'] == date]
    date_filtered.reset_index(inplace = True)
    date_filtered.drop('index',axis = 1, inplace = True)
    
    new_row = {'date': date,'stores':stores['store_name'][j],'store_id':stores['store_id'][j],'total signals' : 0,'unique visitors':0}
    i = 0
    visitors = []
    
    while i < len(date_filtered['point']):

      if (stores['polygon_shape'][j].contains(date_filtered['point'][i])):

        if date_filtered['device_id'][i] not in visitors:
          new_row['unique visitors'] +=1
          visitors.append(date_filtered['device_id'][i])

        new_row['total signals'] += 1
        i +=1

      else:
        i +=1
        continue

    if new_row['total signals'] > 0:
      final_data = final_data.append(new_row,ignore_index=True)
        
  print(j)
  print(new_row)
  j+=1

final_data

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


0
{'date': '01/21/2021', 'stores': "McDonald's", 'store_id': 'place_1', 'total signals': 0, 'unique visitors': 0}
1
{'date': '01/21/2021', 'stores': "McDonald's", 'store_id': 'place_2', 'total signals': 0, 'unique visitors': 0}
2
{'date': '01/21/2021', 'stores': "McDonald's", 'store_id': 'place_3', 'total signals': 1, 'unique visitors': 1}
3
{'date': '01/21/2021', 'stores': "McDonald's", 'store_id': 'place_4', 'total signals': 4, 'unique visitors': 1}
4
{'date': '01/21/2021', 'stores': "McDonald's", 'store_id': 'place_5', 'total signals': 0, 'unique visitors': 0}
5
{'date': '01/21/2021', 'stores': "McDonald's", 'store_id': 'place_6', 'total signals': 0, 'unique visitors': 0}
6
{'date': '01/21/2021', 'stores': "McDonald's", 'store_id': 'place_7', 'total signals': 0, 'unique visitors': 0}
7
{'date': '01/21/2021', 'stores': "McDonald's", 'store_id': 'place_8', 'total signals': 0, 'unique visitors': 0}
8
{'date': '01/21/2021', 'stores': "McDonald's", 'store_id': 'place_9', 'total signals':

KeyboardInterrupt: ignored

In [ ]:
final_data

,date,stores,store_id,total signals,unique visitors
0,01/04/2021,McDonald's,place_1,1,1
1,01/06/2021,McDonald's,place_1,1,1
2,01/08/2021,McDonald's,place_1,1,1
3,01/12/2021,McDonald's,place_1,2,1
4,01/16/2021,McDonald's,place_1,1,1
...,...,...,...,...,...
279,01/08/2021,Aldi,place_92,2,2
280,01/04/2021,Aldi,place_93,1,1
281,01/14/2021,Aldi,place_93,2,1
282,01/16/2021,Aldi,place_93,1,1


In [ ]:
final_data.to_csv('/content/final_data.csv')